## Dependencies

In [1]:
#Including required Libraries required
import ipywidgets as wg
import IPython.display as display 
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
from scipy.optimize import fsolve

mpl.rcParams['figure.dpi'] = 120

## Polynomial models for root finding

In [2]:
def Eq_2(X, coef, Y):
    l = 2
    return(coef[l-2]*(X**(l)) + coef[l-1]*(X**(l-1)) + coef[l]*(X**(l-2)))

def Eq_3(X, coef, Y):
    l = 3
    return(coef[l-3]*(X**(l)) + coef[l-2]*(X**(l-1)) + coef[l-1]*(X**(l-2)) + coef[l]*(X**(l-3)))

def Eq_4(X, coef, Y):
    l = 4
    return(coef[l-4]*(X**(l)) + coef[l-3]*(X**(l-1)) + coef[l-2]*(X**(l-2)) + coef[l-1]*(X**(l-3)) + ((coef[l]*(X**(l-4)) - Y)))

def Eq_5(X, coef, Y):
    l = 5
    return(coef[l-5]*(X**(l)) + coef[l-4]*(X**(l-1)) + coef[l-3]*(X**(l-2)) + coef[l-2]*(X**(l-3)) + coef[l-1]*(X**(l-4)) + ((coef[l]*(X**(l-5)))) - Y)

def Eq_6(X, coef, Y):
    l = 6
    return(coef[l-6]*(X**(l)) + coef[l-5]*(X**(l-1)) + coef[l-4]*(X**(l-2)) + coef[l-3]*(X**(l-3)) + coef[l-2]*(X**(l-4)) + coef[l-1]*(X**(l-5)) + ((coef[l]*(X**(l-6))) - Y))
    
def Eq_7(X, coef, Y):   
    l = 7
    return(coef[l-7]*(X**(l)) + coef[l-6]*(X**(l-1)) + coef[l-5]*(X**(l-2)) + coef[l-4]*(X**(l-3)) + coef[l-3]*(X**(l-4)) + coef[l-2]*(X**(l-5)) + coef[l-1]*(X**(l-6)) + ((coef[l]*(X**(l-7))) - Y))

## Function for fitting a smooth curve in equilibrium data

In [3]:
def FitCurve(x,y,X, l):
    coeff = np.polyfit(x,y,l)
    
    if(l == 2):
        Y = coeff[l-2]*(X**(l)) + coeff[l-1]*(X**(l-1)) + coeff[l]*(X**(l-2))
    if(l == 3):
        Y = coeff[l-3]*(X**(l)) + coeff[l-2]*(X**(l-1)) + coeff[l-1]*(X**(l-2)) + coeff[l]*(X**(l-3))  
    if(l == 4):
        Y = coeff[l-4]*(X**(l)) + coeff[l-3]*(X**(l-1)) + coeff[l-2]*(X**(l-2)) + coeff[l-1]*(X**(l-3)) + coeff[l]*(X**(l-4))  
    if(l == 5):
        Y = coeff[l-5]*(X**(l)) + coeff[l-4]*(X**(l-1)) + coeff[l-3]*(X**(l-2)) + coeff[l-2]*(X**(l-3)) + coeff[l-1]*(X**(l-4)) + coeff[l]*(X**(l-5)) 
    if(l == 6):
        Y = coeff[l-6]*(X**(l)) + coeff[l-5]*(X**(l-1)) + coeff[l-4]*(X**(l-2)) + coeff[l-3]*(X**(l-3)) + coeff[l-2]*(X**(l-4)) + coeff[l-1]*(X**(l-5)) + coeff[l]*(X**(l-6))
    if(l == 7):
        Y = coeff[l-7]*(X**(l)) + coeff[l-6]*(X**(l-1)) + coeff[l-5]*(X**(l-2)) + coeff[l-4]*(X**(l-3)) + coeff[l-3]*(X**(l-4)) + coeff[l-2]*(X**(l-5)) + coeff[l-1]*(X**(l-6)) + ((coeff[l]*(X**(l-7))))

    return coeff, Y

## Function to calculate x coordinate based on the corresponding y coordinate

In [4]:
def Get_X(Y, coeff, x_samp,order):
    index = int(Y*100)
    init_gues = x_samp[index]
    
    if (order == 2):
        DumMy_X = fsolve(Eq_2, init_gues, (coeff, Y))
    if (order == 3):
        DumMy_X = fsolve(Eq_3, init_gues, (coeff, Y))
    if (order == 4):
        DumMy_X = fsolve(Eq_4, init_gues, (coeff, Y))
    elif (order == 5):
        DumMy_X = fsolve(Eq_5, init_gues, (coeff, Y))
    elif (order == 6):
        DumMy_X = fsolve(Eq_6, init_gues, (coeff, Y))
    elif (order == 7):
        DumMy_X = fsolve(Eq_7, init_gues, (coeff, Y))  
    
    return (DumMy_X)

## Distillation Class

In [5]:
class Distillation():
    # Parameterized Constructor
    def __init__(self,x,y,xd,xf,xw,r,f):
        #Setting Equillibrium Curve
        self.X = x
        self.Y = y

        #Expected concentration of the products
        self.Xd = xd
        self.Xf = xf
        self.Xw = xw

        #Reflux ratio in the column
        self.R = r

        #Molar Feed Rate to the column
        self.F = f

    # Method to fit a smooth in equillibrum curve
    def fitcurve(self,order):
        #Sampling points
        self.x_samp = np.arange(0, 1, 0.01)
        self.coeff, self.y_fit = FitCurve(self.X,self.Y,self.x_samp, order)

    # Method to calculate amount of distillate and bottoms product
    def calculate_DW(self):

        #Variable Matrices
        self.A1 = np.array([[  self.F         ],
                            [  self.F*self.Xf ]])
        self.B1 = np.array([[ 1     , 1        ],
                            [self.Xd, self.Xw  ]])

        #Solving for D and W
        self.sol1 = np.linalg.solve(self.B1, self.A1)

        self.D = self.sol1[0][0]
        self.W = self.sol1[1][0]

    # Method to calculate flowrate of liquid and gas in stripping section
    def calculate_LV(self):
        self.L = self.R * self.D
        self.V = self.L + self.D

        self.Lbar = self.L + self.F
        self.Vbar = self.Lbar - self.W

    # Method to calculate the operating line
    def calculate_op_line(self,q):
        #Rectifying Section
        self.Slope_rect = self.R/(self.R+1)
        self.Inter_rect = self.Xd/(self.R+1)

        if(q == 1):
            self.Feed_X = self.Xf
            self.Feed_Y = (self.Slope_rect*self.Feed_X)+self.Inter_rect
        else:
            #Feed Section
            self.Slope_feed = q/(q-1)
            self.Inter_feed = self.Xf/(q-1)
    
            #Variable Matrices
            self.A2 = np.array([[  self.Inter_feed  ],
                                [ -self.Inter_rect] ])
            self.B2 = np.array([[ self.Slope_feed ,-1 ],
                                [ self.Slope_rect ,-1 ]])

            #Solving for Feed Point
            self.sol2 = np.linalg.solve(self.B2, self.A2)

            self.Feed_X = self.sol2[0][0]
            self.Feed_Y = self.sol2[1][0]

        #Stripping Section
        self.Slope_strip = (self.Feed_Y-self.Xw)/(self.Feed_X-self.Xw)
        self.Inter_strip = -(((self.Feed_Y-self.Xw)*self.Feed_X)/(self.Feed_X-self.Xw))+self.Feed_Y

    # Rectifying Section Operating Line
    def Y_Rec(self,X):
        return((self.Slope_rect*X)+(self.Inter_rect))

    # Stripping Section Operating Line
    def Y_Str(self,X):
        return((self.Slope_strip*X)+(self.Inter_strip))

    # Calculating Equillibrum Stages
    def calculate_eq_stg(self,order):
        self.Y_stg = []
        self.X_stg = [self.Xd]

        self.Xtemp = self.Xd
        self.Ytemp = self.Xd

        self.cnt = 0

        while(self.Xtemp > self.Xw):
            if(self.Xtemp > self.Feed_X):
                if(self.cnt == 0):
                    self.Ytemp = self.Y_Rec(self.Xtemp)
                    self.Y_stg.append(self.Ytemp)  
                else:  
                    self.Ytemp = self.Y_Rec(self.Xtemp)
                    self.Y_stg.append(self.Ytemp[0])
                
                self.Xtemp = Get_X(self.Ytemp, self.coeff,self.x_samp,order) 
                self.X_stg.append(self.Xtemp[0])   
                self.cnt += 1
            else: 
                self.Ytemp = self.Y_Str(self.Xtemp)
                self.Y_stg.append(self.Ytemp[0])
                self.Xtemp = Get_X(self.Ytemp, self.coeff, self.x_samp, order) 
                self.X_stg.append(self.Xtemp[0])   

        self.Y_stg.append(self.Xtemp[0])

    # Graphical Design for distillation Column
    def plot_design(self,order):
        #Plotting Data points
        plt.axes([0,0,1,1])
        plt.plot(self.X,self.Y,'bx',label='x-y Data')
        plt.title('Distillation Column Design (MacCabe-Thiele Method)')
        plt.xlabel('x')
        plt.ylabel('y')

        #Sketching Best fit curve
        plt.plot(self.x_samp, self.y_fit, 'b-',label='Best Fit Curve')
        plt.grid(False)
        plt.plot([0, 1],[0, 1],'b--')
        plt.plot([self.Xd, self.Xf, self.Xw, self.Feed_X],[self.Xd, self.Xf, self.Xw, self.Feed_Y],'go')

        #Sketching Operating Lines
        plt.plot([self.Xd, self.Feed_X, self.Xw],[self.Xd, self.Feed_Y, self.Xw],'g--',label='Operating Lines')
        plt.plot([self.Feed_X, self.Xf],[self.Feed_Y, self.Xf],'g--')

        #Sketching Stages on the Diagram
        for m in range(0,(len(self.X_stg)-1),1):
            plt.plot([self.X_stg[m], self.X_stg[m+1]], [self.Y_stg[m], self.Y_stg[m]], 'ro-')
            if(m != (len(self.X_stg)-2)):
                plt.plot([self.X_stg[m+1], self.X_stg[m+1]], [self.Y_stg[m], self.Y_stg[m+1]], 'ro-')
            else:
                plt.plot([self.X_stg[m+1], self.X_stg[m+1]], [self.Y_stg[m], self.Y_stg[m+1]], 'ro-',label='Theretical Stages')
            
        plt.legend()

## Distillation Column Design

In [6]:
def Design_Dist_Column(xd,xf,xw,r,f,order,q):

    #Accquiring VLE Data
    ''' Mole Fraction of MVC in liquid phase '''
    x = [0, 0.074084725, 0.164856116, 0.272477024, 0.401160775, 0.55633974, 0.745066193, 0.951168053]

    ''' Mole Fraction of MVC in gas phase '''
    y = [0, 0.220202250, 0.415447791, 0.579231452, 0.715550937, 0.828020069, 0.919890023, 0.987372668]

    design_prob = Distillation(x,y,xd,xf,xw,r,f)

    design_prob.fitcurve(order)

    design_prob.calculate_DW()

    design_prob.calculate_LV()

    design_prob.calculate_op_line(q)

    design_prob.calculate_eq_stg(order)

    design_prob.plot_design(order)

## Interactive Cell

In [7]:
wg.interact(Design_Dist_Column,
            xd=wg.FloatSlider(value=0.9,max=0.99,min=0.7,step=0.01,continuous_update=False),
            xf=wg.FloatSlider(value=0.4,max=0.5,min=0.35,step=0.05,continuous_update=False),
            xw=wg.FloatSlider(value=0.1,max=0.2,min=0.01,step=0.01,continuous_update=False),
            r=wg.FloatSlider(value=3,max=10,min=1.2,step=0.5,continuous_update=False),
            f=wg.IntSlider(value=10,max=100,min=1,step=10,continuous_update=False),
            order=wg.IntSlider(value=5,max=7,min=2,step=1,continuous_update=False),
            q=wg.FloatSlider(value=0.5,max=2,min=0,step=0.1,continuous_update=False))

interactive(children=(FloatSlider(value=0.9, continuous_update=False, description='xd', max=0.99, min=0.7, ste…

<function __main__.Design_Dist_Column(xd, xf, xw, r, f, order, q)>